In [3]:
!pip install transformers datasets torch nltk spacy sentencepiece
!pip install python -m spacy download en_core_web_sm


  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 5.9 MB/s eta 0:00:00
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
   ---------------------------------------- 0.0/28.0 MB ? eta -:--:--
    --------------------------------------- 0.5/28.0 MB 2.5 MB/s eta 0:00:11
   - -------------------------------------- 1.3/28.0 MB 3.4 MB/s eta 0:00:08
   --- ------------------------------------ 2.6/28.0 MB 4.6 MB/s eta 0:00:06
   ----- ---------------------------------- 3.9/28.0 MB 5.1 MB/s eta 0:00:05
   ------ --------------------------------- 4.7/28.0 MB 5.0 MB/s eta 0:00:05
   ------- -------------------------------- 5.2/28.0 MB 4.6 MB/s eta 0:00:05
   --------- ------------------------------ 6.8/28.0 MB 4.8 MB/s eta 0:00:05
   ----------- ---------------------------- 8.4/28.0 MB 5.1 MB/s eta 0:00:04
   ------------- ------------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -m


In [7]:
import pandas as pd

# Load Excel
df = pd.read_excel("NewsSumm Dataset.xlsx")

# Inspect columns (VERY IMPORTANT)
print(df.columns)

# Save as CSV
df.to_csv("newssuumm.csv", index=False)



Index(['newspaper_name', 'published_date\n', 'headline', 'article_text',
       'human_summary', 'news_category'],
      dtype='object')


In [14]:
from datasets import load_dataset
import nltk
nltk.download("punkt")

# 1️⃣ Load dataset
dataset = load_dataset(
    "csv",
    data_files={"train": "newssuumm.csv"}
)

# 2️⃣ Filter bad rows
dataset = dataset.filter(
    lambda x: x["article_text"] is not None
    and isinstance(x["article_text"], str)
    and len(x["article_text"].strip()) > 0
)

# 3️⃣ Preprocess function (IMPORT INSIDE FUNCTION ✅)
def preprocess(example):
    from nltk.tokenize import sent_tokenize  # 🔥 REQUIRED for num_proc

    text = example["article_text"]

    if text is None or not isinstance(text, str) or text.strip() == "":
        example["sentences"] = []
    else:
        example["sentences"] = sent_tokenize(text)

    return example

# 4️⃣ Parallel mapping
dataset = dataset.map(
    preprocess,
    num_proc=4,   # reduce to 2 if needed
    desc="Tokenizing articles into sentences"
)

# 5️⃣ Verify
print(dataset["train"][0].keys())
print(dataset["train"][0]["sentences"][:3])



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Tokenizing articles into sentences (num_proc=4): 100%|██████████| 348726/348726 [00:52<00:00, 6652.61 examples/s]

dict_keys(['newspaper_name', 'published_date\n', 'headline', 'article_text', 'human_summary', 'news_category', 'sentences'])
['Prime Minister Narendra Modi Monday hailed the contribution of medical workers during the coronavirus pandemic.', 'Addressing an event at the Rajiv Gandhi Health University in Bengaluru via video conference, Modi said, “The virus may be an invisible enemy.', 'But our warriors, medical workers are invincible.']


In [16]:
import torch
from transformers import (
    PegasusTokenizer,
    PegasusForConditionalGeneration,
    pipeline
)
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize



c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
SUM_MODEL = "google/pegasus-cnn_dailymail"

sum_tokenizer = PegasusTokenizer.from_pretrained(SUM_MODEL)
sum_model = PegasusForConditionalGeneration.from_pretrained(SUM_MODEL)


c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpo

In [19]:
from transformers import pipeline
import torch

nli_pipeline = pipeline(
    task="text-classification",
    model="roberta-large-mnli",
    framework="pt",  # 🔥 FORCE PYTORCH
    device=0 if torch.cuda.is_available() else -1
)



Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [20]:
def generate_summary(article, max_length=128):
    inputs = sum_tokenizer(
        article,
        truncation=True,
        padding="longest",
        return_tensors="pt"
    )

    with torch.no_grad():
        summary_ids = sum_model.generate(
            **inputs,
            num_beams=4,
            max_length=max_length
        )

    return sum_tokenizer.decode(
        summary_ids[0],
        skip_special_tokens=True
    )


In [25]:
def check_claim_against_article(claim, article_sentences, threshold=0.7):
    """
    Returns:
    - supported (True / False)
    - best_support_sentence
    - entailment_score
    """
    best_score = 0.0
    best_sentence = None

    for sent in article_sentences:
        result = nli_pipeline({
            "text": sent,
            "text_pair": claim
        })

        # 🔥 Handle both dict and list outputs safely
        if isinstance(result, list):
            result = result[0]

        label = result["label"]
        score = result["score"]

        if label == "ENTAILMENT" and score > best_score:
            best_score = score
            best_sentence = sent

    if best_score >= threshold:
        return True, best_sentence, best_score
    else:
        return False, None, best_score


In [26]:
def hallucination_analyzer(article, summary):
    article_sentences = sent_tokenize(article)
    summary_sentences = sent_tokenize(summary)

    analysis = []

    for claim in summary_sentences:
        supported, support_sentence, score = check_claim_against_article(
            claim, article_sentences
        )

        analysis.append({
            "claim": claim,
            "supported": supported,
            "supporting_sentence": support_sentence,
            "entailment_score": round(score, 3)
        })

    return analysis


In [27]:
def hallucination_risk_level(analysis):
    supported = sum(1 for a in analysis if a["supported"])
    total = len(analysis)

    ratio = supported / total if total > 0 else 0

    if ratio >= 0.85:
        return "Low"
    elif ratio >= 0.6:
        return "Medium"
    else:
        return "High"


In [28]:
article = dataset["train"][0]["article_text"]

summary = generate_summary(article)
analysis = hallucination_analyzer(article, summary)
risk = hallucination_risk_level(analysis)

print("SUMMARY:\n", summary)
print("\nHALLUCINATION RISK:", risk)

for a in analysis:
    print(a)


SUMMARY:
 Prime Minister Narendra Modi Monday hailed the contribution of medical workers during the coronavirus pandemic .<n>He also said that the world is looking at them with gratitude, hope, and seeks both ‘care’ and ‘cure’<n>During the address, PM Modi also stressed that violence against COVID warriors will not be tolerated .

HALLUCINATION RISK: High
{'claim': 'Prime Minister Narendra Modi Monday hailed the contribution of medical workers during the coronavirus pandemic .<n>He also said that the world is looking at them with gratitude, hope, and seeks both ‘care’ and ‘cure’<n>During the address, PM Modi also stressed that violence against COVID warriors will not be tolerated .', 'supported': False, 'supporting_sentence': None, 'entailment_score': 0.0}
